In [1]:
import pandas as pd
import json
import requests
from config import api_key
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from flask import Flask, jsonify, render_template

In [2]:
def get_businesses(location, term, api_key):
    headers = {'Authorization': 'Bearer %s' % api_key}
    url = 'https://api.yelp.com/v3/businesses/search'

    data = []
    for offset in range(0, 1000, 50):
        params = {
            'limit': 50, 
            'location': location.replace(' ', '+'),
            'term': term.replace(' ', '+'),
            'offset': offset
        }

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data += response.json()['businesses']
        elif response.status_code == 400:
            print('400 Bad Request')
            break

    return data

In [3]:
#location can be entered using city name or zipcode
location = "San Francisco"
term = "restaurant"
response = get_businesses(location, term, api_key)

In [4]:
business_name = []
business_review_count = []
avg_rating = []
lat = []
long = []
address = []
phone = []
alias = []
category = []

for result in response:
    business_name.append(result["name"])
    business_review_count.append(result["review_count"])
    avg_rating.append(result["rating"])
    lat.append(result["coordinates"]["latitude"])
    long.append(result["coordinates"]["longitude"])
    address.append(result["location"]["display_address"][0])
    phone.append(result["display_phone"])
    alias.append(result["categories"][0]["alias"])
    category.append(result["categories"][0]["title"])

In [5]:
result_df = pd.DataFrame({ "Business_Name" : business_name,
                           "Review_Count" : business_review_count,
                           "Average_Rating" : avg_rating,
                           "Alias" : alias,
                           "Category" : category,
                           "Latitude" : lat,
                           "Longtitude" : long,
                           "Address" : address,
                           "Phone_Number" : phone })
result_df = result_df.astype({"Business_Name": str, "Review_Count": int, "Average_Rating": float,
                              "Alias": str, "Category": str, "Latitude": float, "Longtitude": float, 
                              "Address":str, "Phone_Number": str})
result_df

,Business_Name,Review_Count,Average_Rating,Alias,Category,Latitude,Longtitude,Address,Phone_Number
0,El Fuego,20,5.0,tacos,Tacos,37.764910,-122.400650,17TH St And Carolina St,(415) 465-9395
1,Fog Harbor Fish House,6069,4.0,seafood,Seafood,37.809041,-122.410233,The Embarcadero,(415) 421-2442
2,Chicken as Cluck,104,4.0,chickenshop,Chicken Shop,37.750586,-122.395178,1760 Cesar Chavez St,(415) 670-0428
3,San Ho Won,17,5.0,korean,Korean,37.759440,-122.410170,2170 Bryant St,
4,San Tung,6872,4.0,chinese,Chinese,37.763693,-122.469008,1031 Irving St,(415) 242-0828
...,...,...,...,...,...,...,...,...,...
995,Dragon Well,515,4.0,chinese,Chinese,37.800720,-122.438680,2142 Chestnut St,(415) 474-6888
996,Pho Phu Quoc PPQ Beef Noodle House Restaurant,1369,3.5,vietnamese,Vietnamese,37.763760,-122.477480,1816 Irving St,(415) 661-8869
997,Thai Idea Vegetarian,743,4.5,thai,Thai,37.783494,-122.419057,710 Polk St,(415) 440-8344
998,Birba,117,4.5,wine_bars,Wine Bars,37.777870,-122.424210,458 Grove St,(415) 549-7612


In [6]:
result_df["Category"].value_counts()

Italian           71
American (New)    62
Thai              50
Mexican           43
Pizza             43
                  ..
Desserts           1
Sports Clubs       1
Arcades            1
Fondue             1
Irish              1
Name: Category, Length: 123, dtype: int64

In [7]:
engine = create_engine("sqlite:///sf_restaurant_db.sqlite")
conn = engine.connect()
Base = declarative_base()

In [8]:
class Restaurant(Base):
    __tablename__ = "restaurant_info"
    __table_args__ = {'extend_existing': True} 
    id = Column(Integer, primary_key = True)
    Business_Name = Column(String(50))
    Review_Count = Column(Integer)
    Average_Rating = Column(Float)
    Alias = Column(String(30))
    Category = Column(String(30))
    Latitude = Column(Float)
    Longtitude = Column(Float)
    Address = Column(String(200))
    Phone_Number = Column(String(30))

In [9]:
Base.metadata.create_all(conn)

In [10]:
result_df.to_sql("restaurant_info", con=engine, if_exists='replace', index_label='id')

In [11]:
restaurant_data = pd.read_sql_query('select * from restaurant_info', con=engine)
restaurant_data

,id,Business_Name,Review_Count,Average_Rating,Alias,Category,Latitude,Longtitude,Address,Phone_Number
0,0,El Fuego,20,5.0,tacos,Tacos,37.764910,-122.400650,17TH St And Carolina St,(415) 465-9395
1,1,Fog Harbor Fish House,6069,4.0,seafood,Seafood,37.809041,-122.410233,The Embarcadero,(415) 421-2442
2,2,Chicken as Cluck,104,4.0,chickenshop,Chicken Shop,37.750586,-122.395178,1760 Cesar Chavez St,(415) 670-0428
3,3,San Ho Won,17,5.0,korean,Korean,37.759440,-122.410170,2170 Bryant St,
4,4,San Tung,6872,4.0,chinese,Chinese,37.763693,-122.469008,1031 Irving St,(415) 242-0828
...,...,...,...,...,...,...,...,...,...,...
995,995,Dragon Well,515,4.0,chinese,Chinese,37.800720,-122.438680,2142 Chestnut St,(415) 474-6888
996,996,Pho Phu Quoc PPQ Beef Noodle House Restaurant,1369,3.5,vietnamese,Vietnamese,37.763760,-122.477480,1816 Irving St,(415) 661-8869
997,997,Thai Idea Vegetarian,743,4.5,thai,Thai,37.783494,-122.419057,710 Polk St,(415) 440-8344
998,998,Birba,117,4.5,wine_bars,Wine Bars,37.777870,-122.424210,458 Grove St,(415) 549-7612


In [12]:
session = Session(bind=engine) 
best_restaurant = session.query(Restaurant).filter(Restaurant.Average_Rating >= 4).filter(Restaurant.Review_Count > 2000).all()

In [13]:
for item in best_restaurant:
    print(item.Business_Name)

Fog Harbor Fish House
San Tung
Marufuku Ramen SF
Sotto Mare Oysteria & Seafood
Burma Superstar
The Codmother Fish & Chips
The House
Kokkari Estiatorio
Hunan Home's Restaurant
The Front Porch
Farmhouse Kitchen Thai Cuisine
Cha Cha Cha
Limón Rotisserie
Wayfare Tavern
Hard Knox Cafe
Hog Island Oyster Co
Gary Danko
Mensho Tokyo
Swan Oyster Depot
Kin Khao
Thanh Long
Woodhouse Fish Company
Hops & Hominy
House of Prime Rib
Golden Boy Pizza
Coqueta
El Farolito
Greens Restaurant
Nopa
La Taqueria
Suppenküche
Zuni Café
Brenda's French Soul Food
Chez Maman East
Pearl's Deluxe Burgers
Delfina
The Monk's Kettle
La Mar Cebichería Peruana
Izakaya Sozai
Marlowe
Mandalay
Tony's Pizza Napoletana
Little Star Pizza
Tommy's Joynt
Perbacco
Boudin Bakery & Cafe
HRD
Tadich Grill
Pancho Villa Taqueria
Nick's Crispy Tacos
Lers Ros
Amber India
Scoma's Restaurant
Gracias Madre
Chapeau!
DOSA on Fillmore
Liholiho Yacht Club
Good Mong Kok Bakery
Pier Market Seafood Restaurant
Saigon Sandwich
Loló
State Bird Provision